In [ ]:
#import modules
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Data Preprocessing and Image Conversion
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 10

In [ ]:
#Image Conversion
import os
from PIL import Image

In [ ]:
# Function to convert images and replace them
def convert_images(folder_path):
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        if file_name.lower().endswith((".png", ".jpg", ".jpeg")):
            file_path = os.path.join(folder_path, file_name)

            try:
                image = Image.open(file_path)
            except Exception as e:
                print(f"Error opening {file_name}: {str(e)}")
                continue

            new_file_path = os.path.splitext(file_path)[0] + ".jpg"

            try:
                image.save(new_file_path, "JPEG")
            except Exception as e:
                print(f"Error converting {file_name} to JPEG: {str(e)}")
                continue

            os.remove(file_path)
            os.rename(new_file_path, file_path)
            print(f"{file_name} converted to JPEG and replaced successfully.")

In [ ]:
folder_path = "/content/drive/MyDrive/Modified data"

In [ ]:
# Convert images in  dataset folder
convert_images(folder_path)

In [ ]:
# Importing Dataset
dataset_xray = tf.keras.preprocessing.image_dataset_from_directory(
    folder_path,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 3094 files belonging to 3 classes.


In [ ]:
# Data Preperation
for image_batch, label_batch in dataset_xray.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

(32, 224, 224, 3)
[1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [ ]:
# Data Exploration
plt.figure(figsize=(7, 7))
for image_batch, label_batch in dataset_xray.take(1):
    for i in range(12):
        if not tf.reduce_all(tf.equal(label_batch, 0)):
            print("Error: Label batch is empty.")
            break
        if i >= len(label_batch):
            print("Error: Index out of bounds for label batch.")
            break
        ax = plt.subplot(3, 4, i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

Error: Label batch is empty.


<Figure size 700x700 with 0 Axes>

In [ ]:
# Define a function to get dataset partitions
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1,
                              shuffle=True, shuffle_size=10000):
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    remaining_ds = ds.skip(train_size)
    val_ds = remaining_ds.take(val_size)
    test_ds = remaining_ds.skip(val_size)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset_xray)
print(len(train_ds), len(val_ds), len(test_ds))

77 9 11


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),  # Remove 'experimental'
    layers.Rescaling(1.0 / 255)
])

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),  # Remove 'experimental'
    layers.Rescaling(0.2),
    layers.RandomContrast(factor=0.2),  # Remove 'experimental'
    layers.RandomZoom(0.2)  # Remove 'experimental'
])

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [ ]:


# URL of the pretrained ViT model from TensorFlow Hub
vit_model_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"

# Load the pretrained ViT model
vit_layer = hub.KerasLayer(vit_model_url, trainable=False)

def build_vit_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Preprocess the inputs if required
    x = inputs

    # Extract features using the pretrained ViT model
    x = vit_layer(x)

    # Add a fully connected layer for classification
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Define the CNN Model
input_shape = (224, 224, CHANNELS)
n_classes = 3

# Build the model
model = build_vit_model(input_shape, n_classes)

# Create a sample input tensor (replace with actual data)
sample_input = tf.random.normal((1,) + input_shape)

# Pass the sample input to the model to trigger the build process
_ = model(sample_input)

# Print the model summary
model.summary()

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=None, name=keras_tensor_1>
  • training=None

In [ ]:
# Compile the Model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(patience=3)

In [ ]:
# Training the Model
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)